<a href="https://colab.research.google.com/github/FatemehPasban/Loanword_Identification/blob/main/testing_phase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.1MB/s 
     |████████████████████████████████| 3.3MB 29.5MB/s 
     |████████████████████████████████| 901kB 43.8MB/s 


In [ ]:
import numpy as np

import torch
from torch import nn
import transformers

pickle did not save the model class itself.so:

In [ ]:
class RootClassifierModel(nn.Module):
    

    def __init__(self, config):
        super(RootClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(model_name,return_dict=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.Root_classifier = nn.Linear(config.hidden_size, 2 )
    
    def forward(self, input_ids, attention_mask):
    #def forward(self, input):
        sequence_output , pooled_output = self.bert(
            #input
            input_ids=input_ids, 
            attention_mask=attention_mask
            )
        pooled_output = self.dropout(pooled_output)
        Root_logits = self.Root_classifier(pooled_output)

        return Root_logits

In [ ]:
model_path = '/content/drive/MyDrive/uni2_project/Root Finder Model/RootClassifierModel_bigdataa.pt'
tokenizer = transformers.BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
Root_finder_model = torch.load(model_path)
Root_finder_model.eval()

In [ ]:
def encode_text(tokenizer, text_sequences, max_length):
    token_ids = np.zeros(shape=(len(text_sequences), max_length),
                         dtype=np.int32)
    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded
    attention_masks = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_masks": attention_masks}


In [ ]:
text = input('Enter your word : ')
text1 = list( text)
encoded = encode_text(tokenizer , text1 , 45)
ids = torch.utils.data.DataLoader(encoded['input_ids'] , batch_size= 1 , shuffle=False)
mask = torch.utils.data.DataLoader(encoded['attention_masks'] , batch_size= 1 , shuffle=False)

for id,ms in zip(enumerate(ids),enumerate(mask)):
  result = Root_finder_model(id[1] , ms[1] )

_,pred = torch.max(result,dim=1)
if int(pred)==1 :
  print('ریشه عربی دارد.')
elif int(pred)==0:
  print('ریشه عربی ندارد.')


Enter your word : مثلا
ریشه عربی دارد.


In [ ]:
#Enter your word : استعمال
#.ریشه عربی دارد.

#Enter your word : کلمات
#ریشه عربی دارد.

#Enter your word : اکیدا
#ریشه عربی دارد.

#Enter your word : ممنوع
#ریشه عربی دارد.

#----------
#Enter your word : بکارگیری
#ریشه عربی ندارد.

#Enter your word : واژه
#ریشه عربی ندارد.

#Enter your word : واژگان
#ریشه عربی ندارد.

#Enter your word : عربی   # اشتباه
#ریشه عربی ندارد.

#Enter your word : قدغن
#ریشه عربی ندارد.